In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("LogRegTitanic").getOrCreate()

In [3]:
# df = spark.read.csv('titanic.csv', inferSchema=True, header=True)
df = sqlContext.sql("SELECT * FROM titanic_csv")

In [4]:
df.printSchema()

root
-- PassengerId: integer (nullable = true)
-- Survived: integer (nullable = true)
-- Pclass: integer (nullable = true)
-- Name: string (nullable = true)
-- Sex: string (nullable = true)
-- Age: double (nullable = true)
-- SibSp: integer (nullable = true)
-- Parch: integer (nullable = true)
-- Ticket: string (nullable = true)
-- Fare: double (nullable = true)
-- Cabin: string (nullable = true)
-- Embarked: string (nullable = true)

In [5]:
df.columns

Out[5]: ['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [6]:
my_cols = df.select(['Survived',
 'Pclass','Sex',
 'Age',
 'SibSp',
 'Parch', 'Fare', 'Embarked' ])

In [7]:
my_final_data = my_cols.na.drop()

In [8]:
from pyspark.ml.feature import VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer

In [9]:
gender_indexer = StringIndexer(inputCol='Sex', outputCol="SexIndex")
gender_encoder = OneHotEncoder(inputCol='SexIndex', outputCol='SexVec')


In [10]:
embarked_indexer = StringIndexer(inputCol='Embarked', outputCol="EmbarkedIndex")
embarked_encoder = OneHotEncoder(inputCol='EmbarkedIndex', outputCol='EmbarkedVec')

In [11]:
assembler = VectorAssembler(inputCols=['Pclass', "SexVec", "EmbarkedVec",'Age', 'SibSp', 'Parch', 'Fare'], outputCol='features')

In [12]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [13]:
log_reg_titanic = LogisticRegression(featuresCol='features', labelCol='Survived')

In [14]:
pipeline = Pipeline(stages=[gender_indexer, embarked_indexer, gender_encoder, embarked_encoder, assembler, log_reg_titanic])

In [15]:
train_data, test_data = my_final_data.randomSplit([0.7, 0.3])

In [16]:
fitted_model = pipeline.fit(train_data)

In [17]:
results = fitted_model.transform(test_data)

In [18]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [19]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Survived')

In [20]:
results.select('Survived', 'prediction').show()

+--------+----------+
Survived|prediction|
+--------+----------+
 0| 1.0|
 0| 1.0|
 0| 1.0|
 0| 0.0|
 0| 0.0|
 0| 1.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 1.0|
+--------+----------+
only showing top 20 rows

In [21]:
final_AUC = my_eval.evaluate(results)
final_AUC

Out[22]: 0.7958974358974359